# This is my first kaggle notebook !
# In this notebook i will use the RetinaNet with https://github.com/fizyr/keras-retinanet API and TensorFlow backend

In [ ]:
# First imports and paths
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
path = '/kaggle/input/global-wheat-detection'
trainDir = path + '/train.csv'
trainDataDir = path + '/train/'
testDataDir = path + '/test/'
testDir = path + '/sample_submission.csv'
names = ['ImageID', 'Width', 'Height', 'bbox', 'Source']
data = pd.read_csv(trainDir, skiprows=1,names=names)
print(data.shape)
data

# Data management & Preparations

In [ ]:
# Seperate bboxs to path/to/image.jpg,x1,y1,x2,y2,class_name
dataDF=pd.DataFrame()
dataDF['ImageID']=data['ImageID'].apply(lambda x: f'{trainDataDir}{x}.jpg')

# Fields exteraction
bbox = data.bbox.str.split(",",expand=True)
dataDF['x1'] = bbox[0].str.strip('[').astype(float).apply(np.int)
dataDF['y1'] = bbox[1].str.strip(' ').astype(float).apply(np.int)
dataDF['x2'] = bbox[2].str.strip(' ').astype(float).apply(np.int)+dataDF['x1']
dataDF['y2'] = bbox[3].str.strip(']').astype(float).apply(np.int)+dataDF['y1']
dataDF['class_name'] = 'wheat'
dataDF , dataDF.dtypes

# Visualize the data randomly

In [ ]:
# Viusualise the data with bboxes
import matplotlib.pyplot as plt
import cv2

ndar = np.random.RandomState(50)
def show_images_with_box(df):
    nrows=3
    ncols=3
    fig, axs = plt.subplots(nrows, ncols, figsize=(25, 25), sharex=True, sharey=True)

    for r in range(nrows):
        for c in range(ncols):
            ridx = ndar.choice(range(df.shape[0])) # Random row selection
            img_name = df.iloc[ridx]['ImageID'] # index based data selection
      
            image = plt.imread(img_name) 
                        
            # find all the records of the provided image and draw box on the wheat heads
            chosen_image = df.loc[df["ImageID"]==img_name,["x1","y1","x2","y2"]] # Grabbing the bbox info by label bassed selection
            class_name = 'wheat'
            bbox_array   = np.array(chosen_image.values.tolist())

            for bbox in bbox_array:
                image = cv2.rectangle(image, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color = (255,255,255), thickness=3) 

            axs[r, c].imshow(image)
            axs[r, c].axis('off')
            axs[r, c].set_title(f'#{class_name} marked = {bbox_array.shape[0]}',size='xx-large')
            
              
    plt.suptitle(f'{nrows*ncols} Random images',size='xx-large')
    plt.show() 
    

show_images_with_box(dataDF)

# Clone keras-retinanet ripo & install

In [ ]:
cd /kaggle/input/

In [ ]:
#!git clone https://github.com/fizyr/keras-retinanet.git
!cp -r kerasresnet1/keras-resnet /kaggle/working
!cp -r kerasretinanet1/keras-retinanet /kaggle/working

In [ ]:
%cd /kaggle/working/keras-resnet/

In [ ]:
!pip install . --user --no-deps

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
%cd /kaggle/working/keras-retinanet/

In [ ]:
!pip install . --user --no-deps

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
# Importing necessary libs and functions
import tensorflow as tf
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [ ]:
# Creating the annotations and class files for a valid training as required by RetinaNet 
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'
dataDF.to_csv(ANNOTATIONS_FILE, index=False, header=None)

classes = set(['wheat'])
with open(CLASSES_FILE, 'w') as f: # Open file in 'write' mode
  for i, line in enumerate(sorted(classes)):
    f.write('{},{}\n'.format(line,i))

# Visualize the data as required by RetinaNet
!head classes.csv
!head annotations.csv

In [ ]:
import urllib
import os

os.makedirs("snapshots", exist_ok=True) # Creation of snapshots folder if is not exist
PRETRAINED_MODEL = "./snapshots/_pretrained_model.h5" # Pretrained model name in keras-retinanet ripo
URL_MODEL = "https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5"

#urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL) # Download the model

print('Downloaded pretrained model to ' + PRETRAINED_MODEL) # Notify when done.
!cp -r /kaggle/input/pre-trained-model/ /snapshots

# Train the model OR import a pre-trained one (Training process is long)

In [ ]:
train_preTrain = False # if you want to train switch to 'True' else, False
if train_preTrain:
    !keras_retinanet/bin/train.py \
    --freeze-backbone \
    --random-transform \
    --weights {PRETRAINED_MODEL} \
    --batch-size 16 \
    --steps 500 \
    --epochs 10 \
    csv annotations.csv classes.csv

In [ ]:
!ls snapshots
if train_preTrain:
    model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
else:
    model_path = os.path.join('/kaggle/input/pre-trained-model', sorted(os.listdir('/kaggle/input/pre-trained-model'), reverse=True)[0])
    #model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])

        

print(model_path) # Import a pre-saved trained model

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv(CLASSES_FILE, header=None).T.loc[0].to_dict()

In [ ]:
# Prediction functiom, run images through the model and get the [boxes, scores, labels]
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0)) # exporting predictions on batch

  boxes /= scale # rescale boxes

  return boxes, scores, labels

In [ ]:
# Threshold definition and drawing the rect boxes around the preds
THRES_SCORE = 0.457

def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]): # zipping boxes,scores,labels as one iterator
    if score < THRES_SCORE:
        break

    color = (255,0,0)
    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

In [ ]:
# Plot the detected bboxes over the image
def show_detected_objects(imgName):
  img_path = testDataDir+imgName
  
  image = read_image_bgr(img_path)

  boxes, scores, labels = predict(image)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_detections(draw, boxes, scores, labels)
  plt.figure(figsize=(15,10))
  plt.axis('on')
  plt.imshow(draw)
  plt.show()

In [ ]:
# Run over the test images and get the scores and plot them
imgs=os.listdir(testDataDir) # Get a list with the image's names
for idx in imgs:
    show_detected_objects(idx)

# Preparation for submission

In [ ]:
# Append the predictions by the submission request 
from tqdm.notebook import tqdm # Progress bar for nested loops

preds=[]
imgid=[]
for img in tqdm(imgs,total=len(imgs)):
    pred = ''
    img_id = ''
    img_path = testDataDir+img
    image = read_image_bgr(img_path)
    boxes, scores, labels = predict(image)
    boxes=boxes[0]
    scores=scores[0]
    img_id += (img.split(".")[0])
    for idx in range(boxes.shape[0]):
        if scores[idx]>THRES_SCORE:
            box,score=boxes[idx],scores[idx]
            pred += f"{score:.4f} {int(box[0])} {int(box[1])} {int(box[2]-box[0])} {int(box[3]-box[1])} "
    preds.append(pred)
    imgid.append(img_id)

In [ ]:
print(len(preds))
print(len(imgid))

In [ ]:
submission = {"image_id":imgid, "PredictionString":preds}
submission = pd.DataFrame(submission)
#finalsub = submission.groupby(["image_id"])['PredictionString'].apply(lambda x: ' '.join(x)).reset_index()
#finalsub
submission

In [ ]:
submission.to_csv('/kaggle/working/submission.csv',index=False)